Junjie Zheng

In [1]:
import numpy as np
from scipy.stats import norm

In [2]:
def BLS_Call(S0, K, T, r, sigma):
        if K != 0:
            theta = np.log(K / S0) / (sigma * np.sqrt(T)) + (sigma / 2 - r / sigma) * np.sqrt(T)
            return S0 * norm.cdf(sigma * np.sqrt(T) - theta) - K * np.exp(-r * T) * norm.cdf(-theta)
        else:
            return S0

Simulation of Basket Call Option Prices

In [3]:
def plain_monte_carlo(r, sigma1, sigma2, rho, X0, Y0, T, K, c1, c2, n):
    # Generate correlated normals for Z1 and Z2
    mean = [0, 0]
    cov = [[1, rho], [rho, 1]]
    Z = np.random.multivariate_normal(mean, cov, n)
    Z1 = Z[:, 0]
    Z2 = Z[:, 1]

    # Calculate stock prices at maturity
    X_T = X0 * np.exp((r - 0.5 * sigma1**2) * T + sigma1 * np.sqrt(T) * Z1)
    Y_T = Y0 * np.exp((r - 0.5 * sigma2**2) * T + sigma2 * np.sqrt(T) * Z2)

    # Calculate payoffs
    V_pre = np.maximum(c1 * X_T + c2 * Y_T - K, 0) * np.exp(-r * T)
    V = np.mean(V_pre)
    SE = np.std(V_pre) / np.sqrt(n)

    return V, SE

In [4]:
def method_of_conditioning(r, sigma1, sigma2, rho, X0, Y0, T, K, c1, c2, n):

    sigma_eff = sigma1 * np.sqrt(1 - rho**2)

    # Generate Z2 values
    Z2 = np.random.normal(0, 1, n)
    V_cond = []

    for z in Z2:
        K_eff = np.maximum(K - c2 * Y0 * np.exp((r - 0.5 * sigma2**2) * T + sigma2 * np.sqrt(T) * z), 0)
        S0 = c1 * X0 * np.exp(sigma1 * np.sqrt(T) * rho * z - 0.5 * sigma1**2 * rho**2 * T)
        if K_eff > 0:
          V_cond.append(BLS_Call(S0, K_eff, T, r, sigma_eff))
        else:
          V_cond.append(S0 - np.exp(-r * T))

    V = np.mean(V_cond)
    SE = np.std(V_cond) / np.sqrt(n)

    return V, SE


In [5]:
# Parameters and simulation setup
params = {
    'r': 0.1, 'sigma1': 0.2, 'sigma2': 0.3, 'rho': 0.7,
    'X0': 50, 'Y0': 50, 'T': 1, 'K': 55, 'c1': 0.5, 'c2': 0.5, 'n': 10000
}

# Plain Monte Carlo estimate
mc_estimate, mc_se = plain_monte_carlo(**params)
print(f"Plain Monte Carlo Estimate: {mc_estimate}, Standard Error: {mc_se}")

# Method of Conditioning estimate
cond_estimate, cond_se = method_of_conditioning(**params)
print(f"Method of Conditioning Estimate: {cond_estimate}, Standard Error: {cond_se}")


Plain Monte Carlo Estimate: 4.693198372134479, Standard Error: 0.07832769572872399
Method of Conditioning Estimate: 4.716921394736531, Standard Error: 0.07242773901965338


Estimation of a Vertical Spread under Stochastic Volatility

In [6]:
import numpy as np
from scipy.stats import norm

def euler_scheme_with_control_variates(r, a, Theta, beta, S0, theta0, T, K1, K2, rho, m, n, sigma):
    dt = T / m  # time increment

    # Random numbers for shared use in stock and control variates
    Z = np.random.randn(n, 2 * m)

    W_orig = []
    W_cont = []

    for i in range(n):
        Y_hat = Y_bar = np.log(S0)
        theta = theta0
        for j in range(m):
            Z1 = Z[i][j]
            Z2 = rho * Z1 + np.sqrt(1 - rho**2) * Z[i][j + m]
            Y_hat += (r - 0.5 * theta**2) * dt + theta * np.sqrt(dt) * Z1
            Y_bar += (r - 0.5 * sigma**2) * dt + sigma * np.sqrt(dt) * Z1
            theta += a * (Theta - theta) * dt + beta * np.sqrt(dt) * Z2

        ST_hat = np.exp(Y_hat)
        ST_bar = np.exp(Y_bar)
        payoff_hat = (np.maximum(ST_hat - K1, 0) - np.maximum(ST_hat - K2, 0)) * np.exp(-r * T)
        payoff_bar = np.maximum(ST_bar - 0.5 * (K1 + K2), 0) * np.exp(-r * T)
        control = BLS_Call(S0, 0.5 * (K1 + K2), T, r, sigma)

        W_orig.append(payoff_hat)
        W_cont.append(payoff_bar - control)

    V_orig = np.mean(W_orig)
    SE_orig = np.std(W_orig) / np.sqrt(n)
    b = -np.cov(W_orig, W_cont)[0, 1] / np.var(W_cont)
    V_cont = V_orig + b * np.mean(W_cont)
    SE_cont = np.sqrt(SE_orig**2 + b**2 * np.var(W_cont) / n)

    return V_orig, SE_orig, V_cont, SE_cont


In [7]:
# Model parameters
params = {
    'r': 0.1, 'a': 3, 'Theta': 0.2, 'beta': 0.1, 'S0': 20, 'theta0': 0.25,
    'T': 1, 'K1': 20, 'K2': 22, 'rho': 0.5, 'm': 50, 'n': 10000, 'sigma': 0.2
}

# Simulation
plain_estimate, plain_se, control_estimate, control_se = euler_scheme_with_control_variates(**params)

print(f"Plain Euler Estimate: {plain_estimate}, Standard Error: {plain_se}")
print(f"Controlled Euler Estimate: {control_estimate}, Standard Error: {control_se}")


Plain Euler Estimate: 0.9517050840933918, Standard Error: 0.008486693223387251
Controlled Euler Estimate: 0.9628713880858786, Standard Error: 0.010341126939454336
